In [1]:
import json
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset

In [2]:
# Open the dataset
dataset_train = json.load(open("league_ranked_2020_short_for_testing.json"))

In [3]:
# Open the table with which one can convert champion id to champion name
championid_to_name = pd.read_csv("riot_champion.csv")
championid_to_name.tail(3)

,Unnamed: 0,version,id,key,name,title,blurb,tags,partype,info.attack,...,stats.hpregen,stats.hpregenperlevel,stats.mpregen,stats.mpregenperlevel,stats.crit,stats.critperlevel,stats.attackdamage,stats.attackdamageperlevel,stats.attackspeedperlevel,stats.attackspeed
145,0,10.6.1,Zilean,26,Zilean,the Chronokeeper,"Once a powerful Icathian mage, Zilean became o...","['Support', 'Mage']",Mana,2,...,5.5,0.5,11.335,0.80,0,0,51.640,3.0,2.13,0.625
146,0,10.6.1,Zoe,142,Zoe,the Aspect of Twilight,"As the embodiment of mischief, imagination, an...","['Mage', 'Support']",Mana,1,...,6.5,0.6,8.000,0.65,0,0,58.000,3.3,2.50,0.625
147,0,10.6.1,Zyra,143,Zyra,Rise of the Thorns,"Born in an ancient, sorcerous catastrophe, Zyr...","['Mage', 'Support']",Mana,4,...,5.5,0.5,13.000,0.40,0,0,53.376,3.2,2.11,0.625


# Convert Champion Key into Index Notatation
The Champion key which you can get by `championid_to_name["keys"]` is weird, the Keys are not ascending/descending in regular intervalls.
So instead we convert the Champion key to the index with that The Champion is between 0 and 147.

In [4]:
# Make championid_to_name["key"] into a list
lookuptable = championid_to_name["key"].values.tolist()

In [5]:
# Get example data
data1 = [dataset_train["data"][0][i]["championId"] for i in range(10)]
data1

[7, 350, 266, 517, 110, 235, 81, 127, 84, 421]

In [6]:
# Convert the example data into index notation
data2 = torch.tensor([lookuptable.index(data1[i]) for i in range(10)])
data2

tensor([ 61, 141,   0, 112, 127, 100,  27,  64,   2,  93])

# Convert Champion key into Champion name

In [14]:
championid_to_name.head(3)

,Unnamed: 0,version,id,key,name,title,blurb,tags,partype,info.attack,...,stats.hpregen,stats.hpregenperlevel,stats.mpregen,stats.mpregenperlevel,stats.crit,stats.critperlevel,stats.attackdamage,stats.attackdamageperlevel,stats.attackspeedperlevel,stats.attackspeed
0,0,10.6.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"['Fighter', 'Tank']",Blood Well,8,...,3.0,1.0,0.0,0.0,0,0,60.00,5.0,2.5,0.651
1,0,10.6.1,Ahri,103,Ahri,the Nine-Tailed Fox,Innately connected to the latent power of Rune...,"['Mage', 'Assassin']",Mana,3,...,6.5,0.6,8.0,0.8,0,0,53.04,3.0,2.0,0.668
2,0,10.6.1,Akali,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,['Assassin'],Energy,5,...,8.0,0.5,50.0,0.0,0,0,62.40,3.3,3.2,0.625


In [15]:
championid_to_name.loc[championid_to_name["key"] == data1[8]]["name"]

2    Akali
Name: name, dtype: object

# Convert Data into one hot vector encoding

In [16]:
# Example
torch.eye(150)[data2[8]]

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])

In [17]:
# convert data
test = torch.eye(148).index_select(dim=0, index=data2)
print(test)
print(test.shape)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([10, 148])
